In [ ]:
import open3d as o3d
import pickle 
import time
import numpy as np

import sys
import os

grasp_transfer_path = os.getenv("GRASP_TRANSFER_SOURCE_DIR")
repo_path = os.getenv("REPO_DIR")

sys.path.insert(0,grasp_transfer_path)

from inference import *

object_part_dict = {
    "watering_pot":"handle",
    "watering_cup":"handle",
    "cheezit_box":"cuboid",
    "pringles": "cuboid",
    "bowl": "rim",
    "cup": "rim"    
}

In [ ]:
pcd_type = "watering_pot"
part_name = object_part_dict[pcd_type]

index = 5 # 1 to 5
pcd_name =pcd_type+'_' + str(index)
candidate_for_obj = 16
    
pcd_path = repo_path+'/datasets/pointclouds_real/' 
pcd = o3d.io.read_point_cloud(pcd_path + '/'+pcd_name+'.pcd')

with open(pcd_path +  '/'+pcd_type+'_orig_grasp.pickle', 'rb') as handle:
    candidate_bases = pickle.load(handle)[0] .reshape(1,4,4)   
tik = time.time()
pcd_original = pcd
points = np.asarray(pcd_original.points)

pcd_original_filtered = pcd_original.select_by_index(np.where((points[:,2]) > 0.05)[0])

cl, ind =pcd_original_filtered.remove_radius_outlier(nb_points=32, radius=0.02)
pcd_original_filtered = pcd_original_filtered.select_by_index(ind)
# pcd_original_filtered=pcd_original_filtered.voxel_down_sample(0.005) # This can be used to speedup the method 
mesh_transformation, grasp_pose, embedding,model_transfer = get_grasps_for_pcd(pcd_original_filtered, part_name,
                                                                candidate_bases,candidate_for_obj)
tok = time.time()
print(tok-tik)
geometry_list = list()
geometry_list.append(pcd_original)  
gripper_mesh =  get_gripper_simple_mesh(np.zeros((3,)), np.eye(3), 0.10, 0.05, score=1,
                                        color=None, trimesh_flag=False)
gripper_mesh.scale(0.75,[0,0,0])
gripper_mesh = gripper_mesh.transform(grasp_pose)
geometry_list.append(gripper_mesh)  
o3d.visualization.draw_geometries(geometry_list) 

In [ ]:
# from se3 import SE3
## Reconstruction View
geometry_list = list()
# geometry_list.append(pcd_original)
geometry_list.append(pcd_original_filtered)

gripper_mesh =  get_gripper_simple_mesh(np.zeros((3,)), np.eye(3), 
                                        0.10, 0.03, score=0, color=None, trimesh_flag=False)

gripper_mesh.scale(1,[0,0,0])
gripper_mesh = gripper_mesh.transform(grasp_pose)
embed = embedding
v,t,_ = estimate_mesh_from_model_with_embedding(model_transfer, 2000, 
                                                torch.Tensor(embed).cuda(),
                                                64, scale=0.5)### Updated with embedding input

mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector((mesh_transformation@to_hom_np(v).T).T[:,:3]) ## Not sure
mesh.triangles = o3d.utility.Vector3iVector(t)
mesh.compute_vertex_normals()
mesh.paint_uniform_color([0, 1, 0])

geometry_list.append(mesh)                    

geometry_list.append(gripper_mesh)                    

o3d.visualization.draw_geometries(geometry_list)